### PART 3: SQL

In [1]:
# import libraries

import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)

import sqlite3

Part 3: SQL
Assume we are using these two tables:
customers table:
customer_nbr customer_name
------------ --------------
1 Jim Brown
2 Jeff Gordon
3 Peter Green
4 Julie Peters
orders table:
order_nbr order_date customer_nbr order_amt
----------- ---------- ------------ ----------
1 2008-10-01 1 15.50
2 2008-12-15 2 25.00
3 2009-01-02 1 18.00
4 2009-02-20 3 10.25
5 2009-03-05 1 30.00
What are the problem(s) with this SQL query?
SELECT
customers.customer_name,
SUM(COALESCE(orders.order_amt, 0)) AS total_2009
FROM
customers
LEFT OUTER JOIN orders ON (
customers.customer_nbr = orders.customer_nbr
)
WHERE
orders.order_date >= ‘20090101’
GROUP BY
customers.customer_name

In [2]:
# import csv
cust_df = pd.read_csv('question_3_cust_table.csv')
orders_df = pd.read_csv('question_3_order_table.csv')

In [3]:
# connect to database
conn = sqlite3.connect(':memory:')

# create cursor object
cur = conn.cursor()

In [4]:
# write records stored in the dataframe to SQL database
cust_df.to_sql('customers', con = conn, if_exists='append', index=False)
orders_df.to_sql('orders', con = conn, if_exists='append', index=False)

In [5]:
# SQL query to preview table

cur.execute('''SELECT *
               FROM customers''')                             # sql query
cust_df = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
cust_df.columns = [x[0] for x in cur.description]           # labels dataframe columns
cust_df.head()                                                     # view dataframe

,customer_nbr,customer_name
0,1,Jim Brown
1,2,Jeff Gordon
2,3,Peter Green
3,4,Julie Peters


In [6]:
# SQL query to preview table

cur.execute('''SELECT *
               FROM orders''')                             # sql query
ord_df = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
ord_df.columns = [x[0] for x in cur.description]           # labels dataframe columns
ord_df.head()                                                     # view dataframe

,order_nbr,order_date,customer_nbr,order_amt
0,1,2008-10-01,1,15.50
1,2,2008-12-15,2,25.00
2,3,2009-01-02,1,18.00
3,4,2009-02-20,3,10.25
4,5,2009-03-02,1,30.00


In [ ]:
# incorrect SQL query 
cur.execute('''SELECT customers.customer_name,
                      SUM(COALESCE(orders.order_amt, 0)) AS total_2009
               FROM   customers
               LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
               WHERE orders.order_date >= '20090101'
               GROUP BY customers.customer_name''')                             # sql query
query = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
query.columns = [x[0] for x in cur.description]           # labels dataframe columns
query.head()                                                     # view datafram

In [7]:
# SQL query 
cur.execute('''SELECT customers.customer_name,
                      SUM(COALESCE(orders.order_amt, 0)) AS total_2009
               FROM   customers
               LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
               WHERE orders.order_date >= '2009-01-01'
               GROUP BY customers.customer_name''')                             # sql query
query = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
query.columns = [x[0] for x in cur.description]           # labels dataframe columns
query.head()                                                     # view datafram

,customer_name,total_2009
0,Jim Brown,48.00
1,Peter Green,10.25


In [8]:
# SQL query 
cur.execute('''SELECT customers.customer_name,
                      SUM(orders.order_amt) AS total_2009
               FROM   customers
               LEFT OUTER JOIN orders ON (customers.customer_nbr = orders.customer_nbr)
               WHERE orders.order_date >= '2009-01-01'
               GROUP BY customers.customer_name''')                             # sql query
query = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
query.columns = [x[0] for x in cur.description]           # labels dataframe columns
query.head()                                                     # view datafram

,customer_name,total_2009
0,Jim Brown,48.00
1,Peter Green,10.25


In [ ]:
# date needs to be converted/correctly inputted
# for these tables, coalesce is used when null vaues are present and there aren't any null values
# in these tables, so coalesce may not be neccessary